In [1]:
import pandas as pd
import datapane as dp
import datetime
import pytz

In [2]:
df = pd.read_csv("source/cust_pub4.csv")

dp.DataTable(df.head())

/var/folders/_j/3_75d81x58lcmq4c6pqrpvp80000gn/T/ipykernel_85876/2986687292.py:1: DtypeWarning: Columns (26,34) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("source/cust_pub4.csv")


In [3]:
date_cols = ["Paid at", "Fulfilled at", "Created at"] # "Cancelled at" dropped in wrangler

The original data includes entries up until ...

In [4]:
last_entry = df["Created at"].max()
last_entry

'2020-09-12 05:49:39 -0700'

Let's adjust all the dates such that the latest entry occurs as close the 10.15 AM on the 27th of April (a Thurday) as possible.

First, we'll drop all dates after the most recent 27th of April.

In [5]:
df = df[(df["Created at"] < "2020-04-28")] 

Add three years to the dates

In [6]:
for date_col in date_cols:
    df[date_col] = pd.to_datetime(df[date_col], utc=True)
    df.loc[df[date_col].notnull(),date_col] = df.loc[df[date_col].notnull()][date_col] + pd.offsets.DateOffset(years=3)
    df[date_col] = df[date_col].dt.tz_convert('US/Pacific')

/var/folders/_j/3_75d81x58lcmq4c6pqrpvp80000gn/T/ipykernel_85876/4125730964.py:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[df[date_col].notnull(),date_col] = df.loc[df[date_col].notnull()][date_col] + pd.offsets.DateOffset(years=3)


Check the new latest date

In [7]:
df["Created at"].max()

Timestamp('2023-04-27 23:48:28-0700', tz='US/Pacific')

Convert to PST

Remove all entires after 10.15 AM

In [8]:
df = df[(df["Created at"] < "2023-04-27 10:15")]

In [9]:
df["Created at"].max()

Timestamp('2023-04-27 10:10:46-0700', tz='US/Pacific')

Save to CSV

In [12]:
df.to_csv("cust_pub4_pydata.csv")